<a href="https://colab.research.google.com/github/coldsober-irene/ASSIGNMENTS/blob/main/dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##generate transfer function

In [25]:
import numpy as np
from scipy.signal import ss2tf

class Modeling:
  def __init__(self):
    pass

  def matrices(self,width = 8, breadth = 4, k1 = 14.3, k2 = 14.3, k3 = 20, sequence = None):
    A, B, c = (None, None, None)
    area = width * breadth # cm
    r_t = 1
    if sequence == 'h2p1p2':
      A = [[-(k1 + k3)/area, k3 / area],
            [k3 / area, -(k3 + k2)/area]]

      B = [[1/area, 0],[0, 1/ area]]
      C = [0, 1]
      r_t = [[1], [1]]

    elif sequence == 'h1p1p2':
      A = [[-(k1 + k3)/area, k3 / area],
            [k3 / area, -(k3 + k2)/area]]

      B = [[1/area, 0],[0, 1/ area]]
      C = [1, 0]

    elif sequence == 'h2p1':
      A = [[-(k1 + k3)/area, k3 / area],
            [k3 / area, -(k3 + k2)/area]]

      B = [[1/area],[0]]
      C = [0, 1]

    elif sequence == 'h1p1':
      A = [[-(k1 + k3)/area, k3 / area],
            [k3 / area, -(k3 + k2)/area]]

      B = [[1/area],[0]]
      C = [1, 0]

    elif sequence == 'h2p2':
      A = [[-(k1 + k3)/area, k3 / area],
            [k3 / area, -(k3 + k2)/area]]

      B = [[0],[1/area]]
      C = [0, 1]

    elif sequence == 'h1p2':
      A = [[-(k1 + k3)/area, k3 / area],
            [k3 / area, -(k3 + k2)/area]]

      B = [[0],[1/area]]
      C = [1, 0]


    print("A: {}".format(np.array(A)))
    print()
    print("B: {}".format(np.array(B)))
    print()
    print("C: {}".format(np.array(C)))
    print()
    print("r(t): {}".format(r_t))
    print()



In [18]:
model = Modeling()

control h2 by both p1 and p2


In [19]:
model.matrices(sequence = 'h2p1p2')

A: [[-1.071875  0.625   ]
 [ 0.625    -1.071875]]

B: [[0.03125 0.     ]
 [0.      0.03125]]

C: [0 1]

r(t): [[1], [1]]



control h1 by both p1 and p2

In [20]:
model.matrices(sequence = 'h1p1p2')

A: [[-1.071875  0.625   ]
 [ 0.625    -1.071875]]

B: [[0.03125 0.     ]
 [0.      0.03125]]

C: [1 0]

r(t): 1



control h2 by p2

In [21]:
model.matrices(sequence = 'h2p2')

A: [[-1.071875  0.625   ]
 [ 0.625    -1.071875]]

B: [[0.     ]
 [0.03125]]

C: [0 1]

r(t): 1



control h1 by p2

In [22]:
model.matrices(sequence = 'h1p2')

A: [[-1.071875  0.625   ]
 [ 0.625    -1.071875]]

B: [[0.     ]
 [0.03125]]

C: [1 0]

r(t): 1



control h2 by p1

In [23]:
model.matrices(sequence = 'h2p1')

A: [[-1.071875  0.625   ]
 [ 0.625    -1.071875]]

B: [[0.03125]
 [0.     ]]

C: [0 1]

r(t): 1



control h1 by p1

In [24]:
model.matrices(sequence = 'h1p1')

A: [[-1.071875  0.625   ]
 [ 0.625    -1.071875]]

B: [[0.03125]
 [0.     ]]

C: [1 0]

r(t): 1

